In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('data/measurements_fmnist_0_new.csv', decimal='.')

timings = df.tail(2)
timings.columns = ['timestamp']
print(timings)

start = timings['timestamp'].min()
end = timings['timestamp'].max()
runtime = end - start

print(runtime)

# Omit the last two rows
power_draws = df.iloc[:-2]
power_draws.columns = ['current']

time_between_measurements = runtime / power_draws.shape[0]

power_draws['time'] = power_draws.index.to_series().apply(lambda index: time_between_measurements * index)
power_draws['timestamp'] = power_draws.index.to_series().apply(lambda index: start + time_between_measurements * index)

# Now df contains the DataFrame without the last two rows
print(power_draws)


fmnist_timestamps = pd.read_csv('fmnist_log.csv')
fmnist_timestamps.columns = ['timestamp', 'label']
timestamps_for_this_run = fmnist_timestamps[(fmnist_timestamps['timestamp'] > start) & (fmnist_timestamps['timestamp'] < end)]
timestamps_for_this_run['time'] = timestamps_for_this_run['timestamp'].apply(lambda timestamp: timestamp - start)

print(timestamps_for_this_run)

         timestamp
2715  1.715604e+09
2716  1.715605e+09
136.06450057029724
       current        time     timestamp
0     129360.0    0.000000  1.715604e+09
1     114390.0    0.050116  1.715604e+09
2     115120.0    0.100232  1.715604e+09
3     113210.0    0.150348  1.715604e+09
4     116900.0    0.200463  1.715604e+09
...        ...         ...           ...
2710  132910.0  135.813921  1.715605e+09
2711  130100.0  135.864037  1.715605e+09
2712  134070.0  135.914153  1.715605e+09
2713  119270.0  135.964269  1.715605e+09
2714  126380.0  136.014385  1.715605e+09

[2715 rows x 3 columns]
        timestamp                 label        time
179  1.715604e+09         start program    2.375327
180  1.715604e+09             load data   22.745661
181  1.715604e+09   finish loading data   22.831925
182  1.715604e+09            save model   23.227977
183  1.715604e+09       done save model   23.297451
184  1.715604e+09         start epoch 0   23.297560
185  1.715604e+09     start phase train   2

/tmp/ipykernel_5900/593330060.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power_draws['time'] = power_draws.index.to_series().apply(lambda index: time_between_measurements * index)
/tmp/ipykernel_5900/593330060.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  power_draws['timestamp'] = power_draws.index.to_series().apply(lambda index: start + time_between_measurements * index)
/tmp/ipykernel_5900/593330060.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [2]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.line(power_draws, x='time', y='current')

fig.add_hline(y=55*1000, line_dash="dot",
              annotation_text="baseline", 
              annotation_position="bottom right")


for index, row in timestamps_for_this_run.iterrows():
    fig.add_vline(x=row['time'], line_width=3, line_color="green", annotation_text=row['label'])


fig.show()


In [3]:
df = pd.read_csv('data/measurements_sleep_10.csv', decimal='.')

timings = df.tail(2)
timings.columns = ['timestamp']
print(timings)

start = timings['timestamp'].min()
end = timings['timestamp'].max()
runtime = end - start

# Omit the last two rows
power_draws = df.iloc[:-2]
power_draws.columns = ['current']

time_between_measurements = runtime / power_draws.shape[0]

power_draws['time'] = power_draws.index.to_series().apply(lambda index: time_between_measurements * index)

# Now df contains the DataFrame without the last two rows
print(power_draws)

         timestamp
1191  1.715599e+09
1192  1.715599e+09
      current       time
0     89880.0   0.000000
1     90250.0   0.050427
2     87080.0   0.100855
3     96520.0   0.151282
4     93460.0   0.201710
...       ...        ...
1186  86960.0  59.807008
1187  87320.0  59.857435
1188  86860.0  59.907862
1189  86550.0  59.958290
1190  86610.0  60.008717

[1191 rows x 2 columns]


/tmp/ipykernel_5900/1684674653.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

